In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import ttest_ind
from math import sqrt
from scipy.stats import chisquare

In [2]:
data = pd.read_csv("Data_Anon.csv", engine='python')
data['Year'] = pd.to_numeric(data['Year'])
data['N_Participants'] = pd.to_numeric(data['N_Participants'])

# 3.1 Descriptives

Sample, Robots, Autonomy

## Sample

Number of papers.

In [3]:
data_1row = data.drop_duplicates('Unique_ID') #create data frame with 1 row per paper
print("Total number of studies: ", len(data_1row.index))

Total number of studies:  171


In [4]:
# Create table of just data from years 2010-2012
early_years = [2010,2011,2012]
is_2010_12 =  data['Year'].isin(early_years)
early_data = pd.DataFrame(data[is_2010_12])

# Create table with just 1 row per paper
early_1row = early_data.drop_duplicates('Unique_ID')
print('Total studies in 2010-2012: ', len(early_1row))

Total studies in 2010-2012:  58


In [5]:
# Create table of just data from years 2017-2019
late_years = [2017,2018,2019]
is_2017_19 =  data['Year'].isin(late_years)
late_data = pd.DataFrame(data[is_2017_19])

# Create table with just 1 row per paper
late_1row = late_data.drop_duplicates('Unique_ID')
print('Total studies in 2017-2019: ', len(late_1row))

Total studies in 2017-2019:  113


## Robots

Top 3 most frequently used robots.

In [6]:
print('2010-2012')
early_1row['Robot'].value_counts()[0:4]

2010-2012


Not reported    7
Robovie         4
Simon           4
Simulation      2
Name: Robot, dtype: int64

In [7]:
print('2017-2019')
late_1row['Robot'].value_counts()[0:4]

2017-2019


Nao             22
Not reported     6
PR2              3
Furhat           3
Name: Robot, dtype: int64

## Autonomy

Number of papers reporting whether robot was autonomous or not.

In [8]:
print('2010-2012')
(early_1row['Autonomy'].value_counts()/len(early_1row))*100

2010-2012


WoZ             29.310345
Autonomous      29.310345
Not reported    22.413793
Mixed           10.344828
Both             1.724138
Name: Autonomy, dtype: float64

In [9]:
print('2017-2019')
(late_1row['Autonomy'].value_counts()/len(late_1row))*100

2017-2019


Autonomous      37.168142
WoZ             19.469027
Not reported    17.699115
Mixed            3.539823
Name: Autonomy, dtype: float64

## Participant Populations

In [10]:
print('2010-2012')
early_1row['Sample'].value_counts()

2010-2012


Students                                                    18
Adults                                                      16
Not reported                                                11
Children                                                     3
Adults & Students                                            3
Older Adults                                                 2
Young adults                                                 1
Adults (AMT)                                                 1
Adults (AMT & Mailing lists)                                 1
Adults & Students (AMT, Craigslist, Uni recruiting site)     1
Children & Adults                                            1
Name: Sample, dtype: int64

In [11]:
print('2017-2019')
late_1row['Sample'].value_counts()

2017-2019


Students                                                                         35
Adults                                                                           15
Not reported                                                                     15
Adults (AMT)                                                                     14
Adults & Students                                                                 8
Children                                                                          7
Adults (AMT US)                                                                   2
Children (ave. 5.63 yrs) & Parents                                                1
Children (5-10 yrs)                                                               1
Clinicians and AMT users                                                          1
Children/Young adults with ASD                                                    1
Children (3-5 yrs)                                                          

In [12]:
students=early_1row[early_1row.Sample.str.contains('Students',case=False)]
print('Percentage of samples containing Students 2010-2012: ', (len(students['Unique_ID'].value_counts())/len(early_1row))*100)
students=late_1row[late_1row.Sample.str.contains('Students',case=False)]
print('Percentage of samples containing Students 2017-2019: ', (len(students['Unique_ID'].value_counts())/len(late_1row))*100)

Percentage of samples containing Students 2010-2012:  37.93103448275862
Percentage of samples containing Students 2017-2019:  38.93805309734513


In [13]:
adults=early_1row[early_1row.Sample.str.contains('Adults',case=False)]
print('Percentage of samples containing Adults 2010-2012: ', (len(adults['Unique_ID'].value_counts())/len(early_1row))*100)
adults=late_1row[late_1row.Sample.str.contains('Adults',case=False)]
print('Percentage of samples containing Adults 2017-2019: ', (len(adults['Unique_ID'].value_counts())/len(late_1row))*100)

Percentage of samples containing Adults 2010-2012:  44.827586206896555
Percentage of samples containing Adults 2017-2019:  43.36283185840708


## Replication

Number of papers that performed a replication study

In [14]:
data_1row.groupby('Year')['Replication'].value_counts()

Year  Replication
2010  No             19
2011  No             18
2012  No             21
2017  No             37
2018  No             39
      Extension       1
2019  No             34
      Paradigm        1
      Yes             1
Name: Replication, dtype: int64

# 3.2 Power Analyses

In [15]:
Power_analyses = pd.DataFrame({'Not reported': [data_1row['Power_Analyses'].value_counts()[0],
                                               early_1row['Power_Analyses'].value_counts()[0],
                                               late_1row['Power_Analyses'].value_counts()[0]],
                                'Reported': [data_1row['Power_Analyses'].value_counts()[1] + data_1row['Power_Analyses'].value_counts()[2],
                                             early_1row['Power_Analyses'].value_counts()[1],
                                             late_1row['Power_Analyses'].value_counts()[2] + late_1row['Power_Analyses'].value_counts()[1]]})

Power_analyses.set_index([pd.Index(['Overall','2010-2012', '2017-2019'])])

,Not reported,Reported
Overall,165,6
2010-2012,57,1
2017-2019,108,5


# 3.3 Effect Size

In [16]:
print("Number of tests where effect size should be reported.")
print("Total number of tests across all studies: ", data['Effect_size_report'].count())
print("Total number of tests in 2010-12 studies: ", early_data['Effect_size_report'].count())
print("Total number of tests in 2017-19 studies: ", late_data['Effect_size_report'].count())

Number of tests where effect size should be reported.
Total number of tests across all studies:  2303
Total number of tests in 2010-12 studies:  801
Total number of tests in 2017-19 studies:  1502


In [17]:
All_noeffect = data['Effect_size_report'].str.contains('Not reported').value_counts()[True]
All_effect = data['Effect_size_report'].str.contains('Not reported').value_counts()[False]
Early_noeffect = early_data['Effect_size_report'].str.contains('Not reported').value_counts()[True]
Early_effect = early_data['Effect_size_report'].str.contains('Not reported').value_counts()[False]
Late_noeffect = late_data['Effect_size_report'].str.contains('Not reported').value_counts()[True]
Late_effect = late_data['Effect_size_report'].str.contains('Not reported').value_counts()[False]

Effect_size_report = pd.DataFrame({"Not Reported": [All_noeffect,
                                                    Early_noeffect,
                                                    Late_noeffect],
                                   "Reported": [All_effect,
                                                Early_effect,
                                                Late_effect],
                                   "% Reported": [(All_effect/data['Effect_size_report'].count())*100,
                                                 (Early_effect/early_data['Effect_size_report'].count())*100,
                                                 (Late_effect/late_data['Effect_size_report'].count())*100]})

Effect_size_report.set_index([pd.Index(['Overall','2010-2012', '2017-2019'])])

,Not Reported,Reported,% Reported
Overall,1845,458,19.887104
2010-2012,670,131,16.354557
2017-2019,1175,327,21.770972


In [18]:
obs = np.array([[131, 327], [670, 1175]]).T
chi2, p, dof, ex = stats.chi2_contingency(obs, correction=True, lambda_=None)

def cramers_stat(confusion_matrix):
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))

result = cramers_stat(obs)

print("Chi Squared = ", chi2)
print("p = ", p)
print("DF = ", dof)
print("Cramer's phi = ", result)

Chi Squared =  9.282667346098952
p =  0.002313323217501211
DF =  1
Cramer's phi =  0.06348767525505854


**Number of studies reporting effect size at least once**

In [19]:
data_all_eOnce=data[~data['Effect_size_report'].isin(['Not reported'])]
data_all_eOnce = data_all_eOnce[pd.notnull(data_all_eOnce['Effect_size_report'])]
data_all_eOnce_1row = data_all_eOnce.drop_duplicates('Unique_ID')

data_early_eOnce=early_data[~early_data['Effect_size_report'].isin(['Not reported'])]
data_early_eOnce = data_early_eOnce[pd.notnull(data_early_eOnce['Effect_size_report'])]
data_early_eOnce_1row = data_early_eOnce.drop_duplicates('Unique_ID')

data_late_eOnce=late_data[~late_data['Effect_size_report'].isin(['Not reported'])]
data_late_eOnce = data_late_eOnce[pd.notnull(data_late_eOnce['Effect_size_report'])]
data_late_eOnce_1row = data_late_eOnce.drop_duplicates('Unique_ID')

In [20]:
All_eOnce = len(data_all_eOnce_1row)
All = len(data_1row)
Early_eOnce = len(data_early_eOnce_1row)
Early = len(early_1row)
Late_eOnce = len(data_late_eOnce_1row)
Late = len(late_1row)

Effect_size_report = pd.DataFrame({"Not Reported": [(All - All_eOnce),
                                                    (Early - Early_eOnce),
                                                    (Late - Late_eOnce)],
                                   "Reported": [All_eOnce,
                                                Early_eOnce,
                                                Late_eOnce],
                                   "% Reported": [(All_eOnce/All)*100,
                                                 (Early_eOnce/Early)*100,
                                                 (Late_eOnce/Late)*100]})

Effect_size_report.set_index([pd.Index(['Overall','2010-2012', '2017-2019'])])

,Not Reported,Reported,% Reported
Overall,122,49,28.654971
2010-2012,46,12,20.689655
2017-2019,76,37,32.743363


In [21]:
obs = np.array([[12, 37], [46, 76]]).T
chi2, p, dof, ex = stats.chi2_contingency(obs, correction=True, lambda_=None)

def cramers_stat(confusion_matrix):
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))

result = cramers_stat(obs)

print("Chi Squared = ", chi2)
print("p = ", p)
print("DF = ", dof)
print("Cramer's phi = ", result)

Chi Squared =  2.16618606413936
p =  0.14107573565246612
DF =  1
Cramer's phi =  0.11255112065656606


**Reporting Means and SDs so we can calculate Effect Size ourselves**

In [22]:
Mean_report = pd.DataFrame({"Not Reported": [data_1row['Mean_reported'].value_counts()[0],
                                             early_1row['Mean_reported'].value_counts()[0],
                                             late_1row['Mean_reported'].value_counts()[0]],
                           "Reported": [data_1row['Mean_reported'].value_counts()[1],
                                        early_1row['Mean_reported'].value_counts()[1],
                                        late_1row['Mean_reported'].value_counts()[1]],
                            "% Reported": [(data_1row['Mean_reported'].value_counts()[1]/len(data_1row))*100,
                                           (early_1row['Mean_reported'].value_counts()[1]/len(early_1row))*100,
                                           (late_1row['Mean_reported'].value_counts()[1]/len(late_1row))*100]})

Mean_report.set_index([pd.Index(['Overall','2010-2012', '2017-2019'])])

,Not Reported,Reported,% Reported
Overall,89,49,28.654971
2010-2012,36,16,27.586207
2017-2019,53,33,29.203540


In [23]:
Sd_report = pd.DataFrame({"Not Reported": [data_1row['SD_reported'].value_counts()[0],
                                             early_1row['SD_reported'].value_counts()[0],
                                             late_1row['SD_reported'].value_counts()[0]],
                           "Reported": [data_1row['SD_reported'].value_counts()[1],
                                        early_1row['SD_reported'].value_counts()[1],
                                        late_1row['SD_reported'].value_counts()[1]],
                            "% Reported": [(data_1row['SD_reported'].value_counts()[1]/len(data_1row))*100,
                                           (early_1row['SD_reported'].value_counts()[1]/len(early_1row))*100,
                                           (late_1row['SD_reported'].value_counts()[1]/len(late_1row))*100]})

Sd_report.set_index([pd.Index(['Overall','2010-2012', '2017-2019'])])

,Not Reported,Reported,% Reported
Overall,96,42,24.561404
2010-2012,39,13,22.413793
2017-2019,57,29,25.663717


# 4 Discussion

## Pre-registered studies

Number of papers reporting pre-registering some aspect of their study.

In [24]:
data_1row.groupby('Year')['Preregistered'].value_counts()

Year  Preregistered
2010  Not reported     19
2011  Not reported     18
2012  Not reported     21
2017  Not reported     37
2018  Not reported     40
2019  Not reported     35
      Yes               1
Name: Preregistered, dtype: int64

## Open Source

Number of papers reporting that their study materials are open-source, and provide links. <br>

Providing data and materials for replication is recommended by Baxter et al., (2016)

In [25]:
data_1row.groupby('Year')['Opensource'].value_counts()

Year  Opensource  
2010  Not reported    19
2011  Not reported    18
2012  Not reported    21
2017  Not reported    37
2018  Not reported    40
2019  Not reported    34
      Yes              2
Name: Opensource, dtype: int64

## Bayesian Statistics

Number of papers which used Bayesian statistics in their analysis. <br>

Using Bayesian statistics is recommended by Baxter et al., (2016)

In [26]:
bayes=data[data.Test.str.contains('Bayesian',case=False)]
bayes['Unique_ID'].value_counts()

201933    17
Name: Unique_ID, dtype: int64